### Set up pipeline with basic imports, data directories, and file basenames.

In [ ]:
import os
from benbiohelpers.DataPipelineManagement.GenomeManager import getGenomeFastaFilePath
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import getExternalDataDirectory as getMutperiodExternalDataDirectory

hg19FastaFilePath = getGenomeFastaFilePath("hg19")
mutperiodHg19Directory = getMutperiodExternalDataDirectory()

liDataDirectory = os.path.join(getDataDir(), "Li_tXR-seq")
liBaseDataFilePath = os.path.join(liDataDirectory, "Li_tXR-seq.bed")

jiang2uM_CellularDataDir = os.path.join(getDataDir(), "Jiang_BPDE_damage_maps", "BEAS-2B_2uM_BPDE_cell_24h")

### Parse the tXR-seq data for mutperiod input

In [ ]:
from mutperiodpy.input_parsing.ParseXRSeq import parseXRSeq, PresetCallParams
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory

if getFilesInDirectory(liDataDirectory, DataTypeStr.mutations + ".bed", searchRecursively = False) is None:
    print("Parsed data not found. Generating...")
    liMutperiodInputFilePaths = parseXRSeq([liBaseDataFilePath], PresetCallParams.BPDE, hg19FastaFilePath)
else:
    liMutperiodInputFilePaths = [getFilesInDirectory(liDataDirectory, DataTypeStr.mutations + ".bed", searchRecursively = False)]
    print(f"Found parsed data at {liMutperiodInputFilePaths[0]}. Continuing.")

### Run the mutperiod pipeline to determine translational and rotational periodicity of the data.
NOTE: This analysis requires that the damage data has already been parsed for mutperiod using the Jiang_damage_maps_nucleosome_periodicity_data_generation notebook.

In [ ]:
from mutperiodpy.RunAnalysisSuite import runAnalysisSuite
nucleosomeMapFilePaths = list()
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_hybrid_nuc_map", "hg19_hybrid_nuc_map.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map", "hg19_LCL_MNase_nuc_map.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_euchromatin",
#                                            "hg19_LCL_MNase_nuc_map_euchromatin.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_heterochromatin",
#                                            "hg19_LCL_MNase_nuc_map_heterochromatin.bed"))
# nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_LCL_MNase_nuc_map_stringent_euchromatin",
#                                            "hg19_LCL_MNase_nuc_map_stringent_euchromatin.bed"))
nucleosomeMapFilePaths.append(os.path.join(mutperiodHg19Directory, "hg19_NHF1_MNase_nuc_map", "hg19_NHF1_MNase_nuc_map.bed"))

nucleosomeMapNames = [os.path.basename(nucleosomeMapFilePath).rsplit('.', 1)[0] for nucleosomeMapFilePath in nucleosomeMapFilePaths]

runAnalysisSuite(liMutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "No Normalization", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)
runAnalysisSuite(liMutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "Singlenuc/Dinuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)
runAnalysisSuite(liMutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "Trinuc/Quadrunuc", customBackgroundDir = None,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)
runAnalysisSuite(liMutperiodInputFilePaths, nucleosomeMapNames,
                 normalizationMethod = "Custom Background", customBackgroundDir = jiang2uM_CellularDataDir,
                 useSingleNucRadius = True, includeLinker = False, useNucGroupRadius = True)

In [ ]:
from mutperiodpy.RunNucleosomeMutationAnalysis import runNucleosomeMutationAnalysis

rawNucleosomeCountsFilePaths = list()
for nucleosomeMapName in nucleosomeMapNames:
    rawNucleosomeCountsFilePaths += getFilesInDirectory(os.path.join(liDataDirectory, nucleosomeMapName), DataTypeStr.rawNucCounts + ".tsv")
runNucleosomeMutationAnalysis(rawNucleosomeCountsFilePaths,
                              outputFilePath = os.path.join(liDataDirectory, "Li_tXR-seq_raw_periodicity_data.tsv"),
                              overridePeakPeriodicityWithExpected = False, alignStrands = True)

normalizedNucleosomeCountsFilePaths = list()
for nucleosomeMapName in nucleosomeMapNames:
    normalizedNucleosomeCountsFilePaths += getFilesInDirectory(os.path.join(liDataDirectory, nucleosomeMapName), DataTypeStr.normNucCounts + ".tsv")
runNucleosomeMutationAnalysis(normalizedNucleosomeCountsFilePaths,
                              outputFilePath = os.path.join(liDataDirectory, "Li_tXR-seq_damage_normalized_periodicity_data.tsv"),
                              overridePeakPeriodicityWithExpected = False, alignStrands = True)